<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Pandas for Exploratory Data Analysis: Joining Data

_Authors: Kevin Markham (DC)_

In [1]:
import pandas as pd

In [2]:
DATA_DIR = "../assets/data"

## Concatenating DataFrames

`pd.concat` sticks DataFrames together, either on top of each other or next to each other.

In [3]:
# Load movie titles
movie_cols = ["movie_id", "title"]
movies_path = f"{DATA_DIR}/movies.tbl"
movies = pd.read_csv(
    movies_path,
    sep="|",
    header=None,
    names=movie_cols,
    usecols=[0, 1],
    encoding="latin-1",
)
movies.head()

movie_id              title
0         1   Toy Story (1995)
1         2   GoldenEye (1995)
2         3  Four Rooms (1995)
3         4  Get Shorty (1995)
4         5     Copycat (1995)

In [4]:
movies.tail()

movie_id                                      title
1677      1678                          Mat' i syn (1997)
1678      1679                           B. Monkey (1998)
1679      1680                       Sliding Doors (1998)
1680      1681                        You So Crazy (1994)
1681      1682  Scream of Stone (Schrei aus Stein) (1991)

In [5]:
# Define a DataFrame with some additional movie titles
movie_ids = [1683, 1684, 1685, 1686, 1687]

more_movies = pd.DataFrame(
    {
        "movie_id": movie_ids,
        "title": [
            "Avengers: Endgame",
            "Captain Marvel",
            "Us",
            "How to Train Your Dragon: The Hidden World",
            "Shazam!",
        ],
    },
    index=[num - 1 for num in movie_ids],
)
more_movies.head()

movie_id                                       title
1682      1683                           Avengers: Endgame
1683      1684                              Captain Marvel
1684      1685                                          Us
1685      1686  How to Train Your Dragon: The Hidden World
1686      1687                                     Shazam!

*Stick the DataFrames on top of each other*

In [6]:
pd.concat([movies, more_movies]).tail(10)

movie_id                                       title
1677      1678                           Mat' i syn (1997)
1678      1679                            B. Monkey (1998)
1679      1680                        Sliding Doors (1998)
1680      1681                         You So Crazy (1994)
1681      1682   Scream of Stone (Schrei aus Stein) (1991)
1682      1683                           Avengers: Endgame
1683      1684                              Captain Marvel
1684      1685                                          Us
1685      1686  How to Train Your Dragon: The Hidden World
1686      1687                                     Shazam!

In [7]:
# Define a DataFrame with running time information for some movies
movie_ids = range(2, 6)

movie_times = pd.DataFrame(
    {"movie_id": movie_ids, "running_mins": [130, 98, 105, 123],},
    index=[num - 1 for num in movie_ids],
)
movie_times

movie_id  running_mins
1         2           130
2         3            98
3         4           105
4         5           123

*Stick the DataFrames next to each other*

In [8]:
pd.concat([movies, movie_times], axis="columns").head(10)

movie_id                                              title  movie_id  \
0         1                                   Toy Story (1995)       NaN   
1         2                                   GoldenEye (1995)       2.0   
2         3                                  Four Rooms (1995)       3.0   
3         4                                  Get Shorty (1995)       4.0   
4         5                                     Copycat (1995)       5.0   
5         6  Shanghai Triad (Yao a yao yao dao waipo qiao) ...       NaN   
6         7                              Twelve Monkeys (1995)       NaN   
7         8                                        Babe (1995)       NaN   
8         9                            Dead Man Walking (1995)       NaN   
9        10                                 Richard III (1995)       NaN   

   running_mins  
0           NaN  
1         130.0  
2          98.0  
3         105.0  
4         123.0  
5           NaN  
6           NaN  
7           NaN  
8           NaN  
9           NaN

**Note:** As illustrated above, `pd.concat` uses the column names or row index values to align DataFrames, rather than column or row positions.

## Joining (Merging) DataFrames

### Movie Reviews Example

"Tidy data" is data that is organized as follows:

- One variable per column
- One observation per row
- One type of observational unit per table

For instance, our table of movies might exist alongside a table of movie reviews.

In [9]:
# Load movie ratings
rating_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings_path = f"{DATA_DIR}/movie_ratings.tsv"
ratings = (
    pd.read_csv(ratings_path, sep="\t", header=None, names=rating_cols)
    .reset_index()
    .rename(columns={"index": "review_id"})
)
ratings.head()

review_id  user_id  movie_id  rating  timestamp
0          0      196       242       3  881250949
1          1      186       302       3  891717742
2          2       22       377       1  878887116
3          3      244        51       2  880606923
4          4      166       346       1  886397596

Now if we want to know the title associated with a particular rating, we match up the rows of the two tables by `movie_id`:

![](../assets/images/inner_join.png)

SQL calls this "matching up" operation a **join**. Pandas has a method called "join," but it also has a more flexible method `.merge()`.

> **Side note:** Excel uses `VLOOKUP` for this kind of operation.

*Merge 'movies' and 'ratings' (inner join on 'movie_id')*

In [10]:
movie_ratings = movies.merge(ratings)
movie_ratings.head()

movie_id             title  review_id  user_id  rating  timestamp
0         1  Toy Story (1995)         24      308       4  887736532
1         1  Toy Story (1995)        454      287       5  875334088
2         1  Toy Story (1995)        957      148       4  877019411
3         1  Toy Story (1995)        971      280       4  891700426
4         1  Toy Story (1995)       1324       66       3  883601324

Equivalent SQL:

```sql
SELECT *
FROM movies
JOIN ratings
ON movie_id;
```

In [11]:
print(movies.shape)
print(ratings.shape)
print(movie_ratings.shape)

(1682, 2)
(100000, 5)
(100000, 6)


Datasets are often stored in tidy format because that format is space-efficient, flexible, and conceptually clean. However, it may require you to do multiple joins to get all of the information you want for your model in one place.

### Selecting Columns to Merge On

*Define another DataFrame with additional movie reviews*

In [12]:
more_ratings = pd.DataFrame(
    {
        "user_id": [712, 214, 86, 1, 14],
        "movie_id": [24, 1012, 1502, 1682, 1684],
        "rating": [3, 1, 2, 2, 5],
        "timestamp": [870069583, 870069683, 870069701, 870069824, 870070800],
    }
)

By default, `merge` matches up rows that have the same values for columns that are common between the two tables.

For instance, `ratings` and `more_ratings` have four columns in common, so `.merge()` will try (and fail) to find rows from the two tables that match on all of those columns.

*Join `more_ratings` to `ratings` on all columns*

In [13]:
ratings.merge(more_ratings)

Empty DataFrame
Columns: [review_id, user_id, movie_id, rating, timestamp]
Index: []

Equivalent SQL:

```sql
SELECT *
FROM ratings
JOIN more_ratings
ON user_id, movie_id, rating, timestamp;
```

You can override this default behavior of `merge` by telling it exactly which columns to use. For instance, we might look for cases where two reviewers gave a movie the same rating.

*Join `more_ratings` to `ratings` on the "movie_id" column*

In [14]:
ratings.merge(more_ratings, on=["movie_id", "rating"])

review_id  user_id_x  movie_id  rating  timestamp_x  user_id_y  \
0        1978         92        24       3    875640448        712   
1        2165         53        24       3    879442538        712   
2        3535        299        24       3    877877732        712   
3        6901         95        24       3    879192542        712   
4       10435        303        24       3    879468047        712   
..        ...        ...       ...     ...          ...        ...   
63      64119        692      1012       1    876953553        214   
64      71833        697      1012       1    882622824        214   
65      76734        847      1012       1    878775729        214   
66      84278        761      1012       1    876190417        214   
67      41727        286      1502       2    877535499         86   

    timestamp_y  
0     870069583  
1     870069583  
2     870069583  
3     870069583  
4     870069583  
..          ...  
63    870069683  
64    870069683  
65    870069683  
66    870069683  
67    870069701  

[68 rows x 7 columns]

Equivalent SQL:

```sql
SELECT *
FROM ratings
JOIN more_ratings
ON movie_id, rating;
```

Notice that `merge` adds suffixes (here "\_x" and "\_y") to the names of the columns from `ratings` and `more_ratings`, respectively, that are common between `ratings` and `more_ratings` but were not used in the join.

You can also tell `merge` to match on columns with different names. For instance, suppose we wanted to join `movies` and `ratings`, but `movies` used "id" in place of "movie_id."

*Uncomment and run the cell below to try to merge `movies` to `ratings` after renaming "movie_id" in movies -- why does it fail?*

In [15]:
# movies.rename(columns={"movie_id": "id"}).merge(ratings)

*Try again, but tell `merge` what columns to join on for each DataFrame*

In [16]:
movies.rename(columns={"movie_id": "id"}).merge(
    ratings, left_on="id", right_on="movie_id"
)

id                                      title  review_id  user_id  \
0         1                           Toy Story (1995)         24      308   
1         1                           Toy Story (1995)        454      287   
2         1                           Toy Story (1995)        957      148   
3         1                           Toy Story (1995)        971      280   
4         1                           Toy Story (1995)       1324       66   
...     ...                                        ...        ...      ...   
99995  1678                          Mat' i syn (1997)      75323      863   
99996  1679                           B. Monkey (1998)      67302      863   
99997  1680                       Sliding Doors (1998)      80394      863   
99998  1681                        You So Crazy (1994)      92329      896   
99999  1682  Scream of Stone (Schrei aus Stein) (1991)      95376      916   

       movie_id  rating  timestamp  
0             1       4  887736532  
1             1       5  875334088  
2             1       4  877019411  
3             1       4  891700426  
4             1       3  883601324  
...         ...     ...        ...  
99995      1678       1  889289570  
99996      1679       3  889289491  
99997      1680       2  889289570  
99998      1681       3  887160722  
99999      1682       3  880845755  

[100000 rows x 7 columns]

Equivalent SQL:

```sql
SELECT *
FROM movies a
JOIN ratings b
ON a.id = b.movie_id;
```

You can also `merge` to join on the indices. For instance, suppose `movie_id` were the index of `movies`.

*Join `ratings` to `movies` with "movie_id" in the index of `movie`*

In [17]:
movies.set_index("movie_id").merge(ratings, left_index=True, right_on="movie_id")

title  review_id  user_id  \
24                              Toy Story (1995)         24      308   
454                             Toy Story (1995)        454      287   
957                             Toy Story (1995)        957      148   
971                             Toy Story (1995)        971      280   
1324                            Toy Story (1995)       1324       66   
...                                          ...        ...      ...   
75323                          Mat' i syn (1997)      75323      863   
67302                           B. Monkey (1998)      67302      863   
80394                       Sliding Doors (1998)      80394      863   
92329                        You So Crazy (1994)      92329      896   
95376  Scream of Stone (Schrei aus Stein) (1991)      95376      916   

       movie_id  rating  timestamp  
24            1       4  887736532  
454           1       5  875334088  
957           1       4  877019411  
971           1       4  891700426  
1324          1       3  883601324  
...         ...     ...        ...  
75323      1678       1  889289570  
67302      1679       3  889289491  
80394      1680       2  889289570  
92329      1681       3  887160722  
95376      1682       3  880845755  

[100000 rows x 6 columns]

### Join Types

You might have noticed that when we used `merge` above, we dropped rows from the original DataFrames that weren't matched. That kind of join is called an "inner join." You can also do an "outer join", where rows that don't have a match are returned with missing values for the fields that would have come from the other table if there had been a match.

![](../assets/images/outer_join.png)

*Do an outer join of `more_ratings` to `movies`*

In [18]:
movies.merge(more_ratings, how="outer")

movie_id                                      title  user_id  rating  \
0            1                           Toy Story (1995)      NaN     NaN   
1            2                           GoldenEye (1995)      NaN     NaN   
2            3                          Four Rooms (1995)      NaN     NaN   
3            4                          Get Shorty (1995)      NaN     NaN   
4            5                             Copycat (1995)      NaN     NaN   
...        ...                                        ...      ...     ...   
1678      1679                           B. Monkey (1998)      NaN     NaN   
1679      1680                       Sliding Doors (1998)      NaN     NaN   
1680      1681                        You So Crazy (1994)      NaN     NaN   
1681      1682  Scream of Stone (Schrei aus Stein) (1991)      1.0     2.0   
1682      1684                                        NaN     14.0     5.0   

        timestamp  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
...           ...  
1678          NaN  
1679          NaN  
1680          NaN  
1681  870069824.0  
1682  870070800.0  

[1683 rows x 5 columns]

Equivalent SQL:

```sql
SELECT *
FROM movies
FULL OUTER JOIN more_ratings
ON movie_id;
```

You can also do a "left merge" that returns all rows from the "left" DataFrame only regardless of whether they have a match.

![](../assets/images/left_join.png)

*Join `more_ratings` to `movies`, keeping all rows from `movies` even if they don't have a match*

In [19]:
movies.merge(more_ratings, how="left")

movie_id                                      title  user_id  rating  \
0            1                           Toy Story (1995)      NaN     NaN   
1            2                           GoldenEye (1995)      NaN     NaN   
2            3                          Four Rooms (1995)      NaN     NaN   
3            4                          Get Shorty (1995)      NaN     NaN   
4            5                             Copycat (1995)      NaN     NaN   
...        ...                                        ...      ...     ...   
1677      1678                          Mat' i syn (1997)      NaN     NaN   
1678      1679                           B. Monkey (1998)      NaN     NaN   
1679      1680                       Sliding Doors (1998)      NaN     NaN   
1680      1681                        You So Crazy (1994)      NaN     NaN   
1681      1682  Scream of Stone (Schrei aus Stein) (1991)      1.0     2.0   

        timestamp  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
...           ...  
1677          NaN  
1678          NaN  
1679          NaN  
1680          NaN  
1681  870069824.0  

[1682 rows x 5 columns]

Equivalent SQL:

```sql
SELECT *
FROM movies
LEFT JOIN more_ratings
ON movie_id;
```

And you can do a "right merge" that returns all rows from the "right" DataFrame only regardless of whether they have a match.

![](../assets/images/right_join.png)

*Join `more_ratings` to `movies`, keeping all rows from `more_ratings` even if they don't have a match*

In [20]:
movies.merge(more_ratings, how="right")

movie_id                                      title  user_id  rating  \
0        24                 Rumble in the Bronx (1995)      712       3   
1      1012                       Private Parts (1997)      214       1   
2      1502                   Naked in New York (1994)       86       2   
3      1682  Scream of Stone (Schrei aus Stein) (1991)        1       2   
4      1684                                        NaN       14       5   

   timestamp  
0  870069583  
1  870069683  
2  870069701  
3  870069824  
4  870070800

Equivalent SQL:

```sql
SELECT *
FROM movies
RIGHT JOIN more_ratings
ON movie_id;
```

![](../assets/images/join_types.png)

**Exercise**

*Time:* 2 min\
*Format:* Individual\
*Post answers:* Yes

In [21]:
# Create some stock price data
openprice = pd.DataFrame(
    {
        "Symbol": ["AAPL", "DHR", "DAL", "AMZN"],
        "OpenPrice": [217.51, 96.54, 51.45, 1703.34],
    }
)
wkhigh = pd.DataFrame(
    {
        "Symbol": ["DAL", "AMZN", "AAPL", "DHR"],
        "52wkHigh": [60.79, 2050.49, 233.47, 110.11],
    }
)
stockname = pd.DataFrame(
    {
        "Symbol": ["AMZN", "DHR", "DAL", "AAPL"],
        "Name": ["Amazon", "Danaher", "Delta Airlines", "Apple"],
    }
)

- Inner join the `openprice` and `wkhigh` DataFrames.

In [22]:
openprice.merge(wkhigh)

Symbol  OpenPrice  52wkHigh
0   AAPL     217.51    233.47
1    DHR      96.54    110.11
2    DAL      51.45     60.79
3   AMZN    1703.34   2050.49

- Inner join the `openprice` and `stockname` DataFrames.

In [23]:
openprice.merge(stockname)

Symbol  OpenPrice            Name
0   AAPL     217.51           Apple
1    DHR      96.54         Danaher
2    DAL      51.45  Delta Airlines
3   AMZN    1703.34          Amazon

- Inner join all three DataFrames.

In [24]:
openprice.merge(stockname).merge(wkhigh)

Symbol  OpenPrice            Name  52wkHigh
0   AAPL     217.51           Apple    233.47
1    DHR      96.54         Danaher    110.11
2    DAL      51.45  Delta Airlines     60.79
3   AMZN    1703.34          Amazon   2050.49

$\blacksquare$

### Keys

In a traditional relational database, each table has a "primary key" that uniquely identifies each row of the table. For instance, our `movies` DataFrame has the `movie_id` column that uniquely identifies each movie.

![](../assets/images/primary_key.png)

`ratings` also has a column `movie_id`, but it is not a primary key for `ratings` -- it has repeated values. Its purpose in `ratings` is to allow us to join `ratings` with `movies`. Because it refers to the primary key of another table, it is called a *foreign key* in `ratings`.

![](../assets/images/foreign_key.png)

### Sales Data Example

Here are the data dictionaries we'll be using for the following exercise:

<ul>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html">Production.Product</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.ProductSubcategory.html">Production.ProductSubcategory</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderHeader.html">Sales.SalesOrderHeader</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderDetail.html">Sales.SalesOrderDetail</a></li>
</ul>

In [25]:
# Load the data
product = pd.read_csv(f"{DATA_DIR}/Production.Product.csv", sep="\t")
product_subcategory = pd.read_csv(
    f"{DATA_DIR}/Production.ProductSubcategory.csv", sep="\t"
)
sales_header = pd.read_csv(
    f"{DATA_DIR}/Sales.SalesOrderHeader.csv", sep="\t", nrows=1000
)
sales_detail = pd.read_csv(
    f"{DATA_DIR}/Sales.SalesOrderDetail.csv", sep="\t", nrows=1000
)

The [Production.ProductSubcategory data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.ProductSubcategory.html) reports a "primary key" `ProductSubcategoryID`.

The [Production.Product data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html) characterizes `ProductSubcategoryID` as a "foreign key" because it corresponds to the primary key of `Production.ProductSubcategory`.

*Join the Production.Product and Production.ProductSubcategory on `ProductSubcategoryID`*

In [26]:
product.merge(
    product_subcategory, on="ProductSubcategoryID", suffixes=("", "_sub")
).head(3)

ProductID                       Name ProductNumber  MakeFlag  \
0        680  HL Road Frame - Black, 58    FR-R92B-58         1   
1        706    HL Road Frame - Red, 58    FR-R92R-58         1   
2        717    HL Road Frame - Red, 62    FR-R92R-62         1   

   FinishedGoodsFlag  Color  SafetyStockLevel  ReorderPoint  StandardCost  \
0                  1  Black               500           375     1059.3100   
1                  1    Red               500           375     1059.3100   
2                  1    Red               500           375      868.6342   

   ListPrice  ... ProductModelID        SellStartDate SellEndDate  \
0     1431.5  ...            6.0  2008-04-30 00:00:00         NaN   
1     1431.5  ...            6.0  2008-04-30 00:00:00         NaN   
2     1431.5  ...            6.0  2011-05-31 00:00:00         NaN   

   DiscontinuedDate                                 rowguid  \
0               NaN  {43DD68D6-14A4-461F-9069-55309D90EA7E}   
1               NaN  {9540FF17-2712-4C90-A3D1-8CE5568B2462}   
2               NaN  {052E4F8B-0A2A-46B2-9F42-10FEBCFAE416}   

                    ModifiedDate ProductCategoryID     Name_sub  \
0  2014-02-08 10:01:36.827000000                 2  Road Frames   
1  2014-02-08 10:01:36.827000000                 2  Road Frames   
2  2014-02-08 10:01:36.827000000                 2  Road Frames   

                              rowguid_sub     ModifiedDate_sub  
0  {5515F857-075B-4F9A-87B7-43B4997077B3}  2008-04-30 00:00:00  
1  {5515F857-075B-4F9A-87B7-43B4997077B3}  2008-04-30 00:00:00  
2  {5515F857-075B-4F9A-87B7-43B4997077B3}  2008-04-30 00:00:00  

[3 rows x 29 columns]

Equivalent SQL (except for the "\_sub" suffix):

```sql
SELECT *
FROM product
INNER JOIN product_subcategory
ON ProductSubcategoryID;
```

**Exercise**

*Time:* 5 mins\
*Format:* Pairs\
*Post answers:* Yes

- Left join `Sales.SalesOrderDetail` (right) to `Sales.SalesOrderHeader` (left) on the relevant primary/foreign key. (Use the data dictionaries to identify the key. There seems to be a mistake in the `Sales.SalesOrderDetail` data dictionary: "SalesOrderId" is only a foreign key in that table and not a primary key.)

In [27]:
sales_header.merge(sales_detail, how="left", on="SalesOrderID").head()

SalesOrderID  RevisionNumber            OrderDate              DueDate  \
0         43659               8  2011-05-31 00:00:00  2011-06-12 00:00:00   
1         43659               8  2011-05-31 00:00:00  2011-06-12 00:00:00   
2         43659               8  2011-05-31 00:00:00  2011-06-12 00:00:00   
3         43659               8  2011-05-31 00:00:00  2011-06-12 00:00:00   
4         43659               8  2011-05-31 00:00:00  2011-06-12 00:00:00   

              ShipDate  Status  OnlineOrderFlag SalesOrderNumber  \
0  2011-06-07 00:00:00       5                0          SO43659   
1  2011-06-07 00:00:00       5                0          SO43659   
2  2011-06-07 00:00:00       5                0          SO43659   
3  2011-06-07 00:00:00       5                0          SO43659   
4  2011-06-07 00:00:00       5                0          SO43659   

  PurchaseOrderNumber   AccountNumber  ...  SalesOrderDetailID  \
0         PO522145787  10-4020-000676  ...                 1.0   
1         PO522145787  10-4020-000676  ...                 2.0   
2         PO522145787  10-4020-000676  ...                 3.0   
3         PO522145787  10-4020-000676  ...                 4.0   
4         PO522145787  10-4020-000676  ...                 5.0   

   CarrierTrackingNumber  OrderQty  ProductID  SpecialOfferID  UnitPrice  \
0           4911-403C-98       1.0      776.0             1.0   2024.994   
1           4911-403C-98       3.0      777.0             1.0   2024.994   
2           4911-403C-98       1.0      778.0             1.0   2024.994   
3           4911-403C-98       1.0      771.0             1.0   2039.994   
4           4911-403C-98       1.0      772.0             1.0   2039.994   

   UnitPriceDiscount LineTotal                               rowguid_y  \
0                0.0  2024.994  {B207C96D-D9E6-402B-8470-2CC176C42283}   
1                0.0  6074.982  {7ABB600D-1E77-41BE-9FE5-B9142CFC08FA}   
2                0.0  2024.994  {475CF8C6-49F6-486E-B0AD-AFC6A50CDD2F}   
3                0.0  2039.994  {04C4DE91-5815-45D6-8670-F462719FBCE3}   
4                0.0  2039.994  {5A74C7D2-E641-438E-A7AC-37BF23280301}   

        ModifiedDate_y  
0  2011-05-31 00:00:00  
1  2011-05-31 00:00:00  
2  2011-05-31 00:00:00  
3  2011-05-31 00:00:00  
4  2011-05-31 00:00:00  

[5 rows x 36 columns]

- Left join `Sales.SalesOrderDetail` (right) to `Sales.SalesOrderHeader` (left) on the relevant primary/foreign key, and left join `Production.Product` to the result on the relevant primary/foreign key.

In [28]:
(
    sales_header.merge(sales_detail, on="SalesOrderID", how="left").merge(
        product, how="left", on="ProductID",
    )
).head()

SalesOrderID  RevisionNumber            OrderDate              DueDate  \
0         43659               8  2011-05-31 00:00:00  2011-06-12 00:00:00   
1         43659               8  2011-05-31 00:00:00  2011-06-12 00:00:00   
2         43659               8  2011-05-31 00:00:00  2011-06-12 00:00:00   
3         43659               8  2011-05-31 00:00:00  2011-06-12 00:00:00   
4         43659               8  2011-05-31 00:00:00  2011-06-12 00:00:00   

              ShipDate  Status  OnlineOrderFlag SalesOrderNumber  \
0  2011-06-07 00:00:00       5                0          SO43659   
1  2011-06-07 00:00:00       5                0          SO43659   
2  2011-06-07 00:00:00       5                0          SO43659   
3  2011-06-07 00:00:00       5                0          SO43659   
4  2011-06-07 00:00:00       5                0          SO43659   

  PurchaseOrderNumber   AccountNumber  ...  ProductLine  Class  Style  \
0         PO522145787  10-4020-000676  ...           M      H      U    
1         PO522145787  10-4020-000676  ...           M      H      U    
2         PO522145787  10-4020-000676  ...           M      H      U    
3         PO522145787  10-4020-000676  ...           M      H      U    
4         PO522145787  10-4020-000676  ...           M      H      U    

   ProductSubcategoryID  ProductModelID        SellStartDate  \
0                   1.0            19.0  2011-05-31 00:00:00   
1                   1.0            19.0  2011-05-31 00:00:00   
2                   1.0            19.0  2011-05-31 00:00:00   
3                   1.0            19.0  2011-05-31 00:00:00   
4                   1.0            19.0  2011-05-31 00:00:00   

           SellEndDate DiscontinuedDate  \
0  2012-05-29 00:00:00              NaN   
1  2012-05-29 00:00:00              NaN   
2  2012-05-29 00:00:00              NaN   
3  2012-05-29 00:00:00              NaN   
4  2012-05-29 00:00:00              NaN   

                                  rowguid                   ModifiedDate  
0  {02935111-A546-4C6D-941F-BE12D42C158E}  2014-02-08 10:01:36.827000000  
1  {7920BC3B-8FD4-4610-93D2-E693A66B6474}  2014-02-08 10:01:36.827000000  
2  {1B486300-7E64-4C5D-A9BA-A8368E20C5A0}  2014-02-08 10:01:36.827000000  
3  {CA74B54E-FC30-4464-8B83-019BFD1B2DBB}  2014-02-08 10:01:36.827000000  
4  {BBFFF5A5-4BDC-49A9-A5AD-7584ADFFE808}  2014-02-08 10:01:36.827000000  

[5 rows x 60 columns]

$\blacksquare$

## Summary

- "Tidy data" is organized as follows:
    - One variable per column
    - One observation per row
    - One type of observational unit per table
- Storing data in tidy format saves space and is conceptually nice, but it might mean that you have to use multiple tables to answer a given question.
- The Pandas `.merge()` method allows you to join tables on common index and column values.